In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score

In [2]:
def calculate_ece(confidences, accuracies, n_bins=10) -> float:
    """
    Calculate the expected calibration error (ECE) given a list of confidence scores (0-1) and accuracy scores (0 or 1).
    """
    df = pd.DataFrame({"conf": confidences, "acc": accuracies}).dropna()

    confidences = torch.tensor(df["conf"].tolist())
    accuracies = torch.tensor(df["acc"].tolist())
    bin_boundaries = torch.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    ece = torch.zeros(1)
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # Calculated |confidence - accuracy| in each bin
        in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
        prop_in_bin = in_bin.float().mean()
        if prop_in_bin.item() > 0:
            accuracy_in_bin = accuracies[in_bin].float().mean()
            avg_confidence_in_bin = confidences[in_bin].mean()
            ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    
    return ece.item()

# Drop NA on an Individual Basis

In [3]:
stats_df = pd.DataFrame()
for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file)
        acc_no_na = df[["extracted_answer", "correct_answer"]].dropna()
        new_row = pd.DataFrame({
            "Model": [file.replace("mmlu_pro_", "").replace("_eval_all_None.csv", "").strip().capitalize()],
            "Missing answer": [df["extracted_answer"].isna().sum()],
            "Missing verbal numerical confidence": [df["verbal_numerical_confidence"].isna().sum()],
            "Missing logit perplexity confidence": [df["logit_perplexity_confidence"].isna().sum()],
            "Missing verbal linguistic confidence": [df["verbal_linguistic_confidence"].isna().sum()],
            "Accuracy": [(df["extracted_answer"] == df["correct_answer"]).mean()],
            "Accuracy without na": [(acc_no_na["extracted_answer"] == acc_no_na["correct_answer"]).mean()],
            "ECE verbal numerical confidence": [calculate_ece(df["verbal_numerical_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE logit perplexity confidence": [calculate_ece(df["logit_perplexity_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE verbal linguistic confidence": [calculate_ece(df["verbal_linguistic_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "AUROC verbal numerical confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]).values, df["verbal_numerical_confidence"].fillna(0).values),
            "AUROC logit perplexity confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["logit_perplexity_confidence"].fillna(0).values),
            "AUROC verbal linguistic confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["verbal_linguistic_confidence"].fillna(0).values),
            "Mean verbal numerical confidence": [df["verbal_numerical_confidence"].mean()],
            "Mean logit perplexity confidence": [df["logit_perplexity_confidence"].mean()],
            "Mean verbal linguistic confidence": [df["verbal_linguistic_confidence"].mean()],
            "Std verbal numerical confidence": [df["verbal_numerical_confidence"].std()],
            "Std logit perplexity confidence": [df["logit_perplexity_confidence"].std()],
            "Std verbal linguistic confidence": [df["verbal_linguistic_confidence"].std()],
        })
        stats_df = pd.concat([stats_df, new_row], ignore_index=True)
    
display(stats_df.sort_values(by="Model", ignore_index=True))

KeyError: 'Model'

# Drop NA Across All Models

In [ ]:
nan_indices = set()

for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file)

        # Find indices where either column has NaN
        indices_with_nan = df[
            df["extracted_answer"].isna() | df["verbal_numerical_confidence"].isna()
        ].index

        # Combine indices across all files
        nan_indices.update(indices_with_nan)

# Convert to sorted list if needed
nan_indices = sorted(nan_indices)
len(nan_indices)

12032

In [ ]:
stats_df = pd.DataFrame()
for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file).drop(index=nan_indices).reset_index(drop=True)
        acc_no_na = df
        new_row = pd.DataFrame({
            "Model": [file.replace("mmlu_pro_", "").replace("_eval_all_None.csv", "").strip().capitalize()],
            "Missing answer": [df["extracted_answer"].isna().sum()],
            "Missing verbal numerical confidence": [df["verbal_numerical_confidence"].isna().sum()],
            "Missing logit perplexity confidence": [df["logit_perplexity_confidence"].isna().sum()],
            "Missing verbal linguistic confidence": [df["verbal_linguistic_confidence"].isna().sum()],
            "Accuracy": [(df["extracted_answer"] == df["correct_answer"]).mean()],
            "Accuracy without na": [(acc_no_na["extracted_answer"] == acc_no_na["correct_answer"]).mean()],
            "ECE verbal numerical confidence": [calculate_ece(df["verbal_numerical_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE logit perplexity confidence": [calculate_ece(df["logit_perplexity_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE verbal linguistic confidence": [calculate_ece(df["verbal_linguistic_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "AUROC verbal numerical confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]).values, df["verbal_numerical_confidence"].fillna(0).values),
            "AUROC logit perplexity confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["logit_perplexity_confidence"].fillna(0).values),
            "AUROC verbal linguistic confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["verbal_linguistic_confidence"].fillna(0).values),
            "Mean verbal numerical confidence": [df["verbal_numerical_confidence"].mean()],
            "Mean logit perplexity confidence": [df["logit_perplexity_confidence"].mean()],
            "Mean verbal linguistic confidence": [df["verbal_linguistic_confidence"].mean()],
            "Std verbal numerical confidence": [df["verbal_numerical_confidence"].std()],
            "Std logit perplexity confidence": [df["logit_perplexity_confidence"].std()],
            "Std verbal linguistic confidence": [df["verbal_linguistic_confidence"].std()],
        })
        stats_df = pd.concat([stats_df, new_row], ignore_index=True)
    
display(stats_df.sort_values(by="Model", ignore_index=True))

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.